In [1]:
from Util import get_openai_api_key
OPEN_API_KEY=get_openai_api_key()

In [2]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [1]:
#pip install pyautogen pytesseract pillow

In [3]:
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager
from PIL import Image
import pytesseract
import os

In [19]:
folder_path = "/Users/naveenudayakumar/Documents/OpenAI/LLM_P3"

In [20]:
def extract_mcq_text(folder_path):
    all_mcqs = ""
    for file in os.listdir(folder_path):
        if file.lower().endswith((".jpeg", ".jpg", ".png")):
            path = os.path.join(folder_path, file)
            try:
                img = Image.open(path)
                text = pytesseract.image_to_string(img)
                all_mcqs += f"\n🖼️ {file}:\n{text.strip()}\n"
            except Exception as e:
                all_mcqs += f"\n❌ Error reading {file}: {e}\n"
    return all_mcqs if all_mcqs else "⚠️ No valid images found in the folder."

In [25]:
#extract_mcq_text()

In [21]:
# User initiates task
user = UserProxyAgent(name="User",human_input_mode="NEVER", code_execution_config=False)

In [22]:

ocr_agent = AssistantAgent(
    name="OCRAgent",
    llm_config=llm_config,
    description="Extracts MCQs from images in a folder",
    code_execution_config=False,
    function_map={"extract_mcq_text": extract_mcq_text}
)



In [23]:
answering_agent = AssistantAgent(
    name="AnsweringAgent",
    llm_config=llm_config,
    description="Answers MCQs with correct option and explanation",
    code_execution_config=False
)

In [25]:
# === GROUP CHAT SETUP ===
group_chat = GroupChat(
    agents=[user, ocr_agent, answering_agent],
    messages=[],
    max_round=10
)

chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

In [26]:
user.initiate_chat(
    chat_manager,
    message=f"""
Hi team. Here's the task:

1. **OCRAgent**: Run the function `extract_mcq_text` on the folder `{folder_path}` and extract MCQ-style questions and options from all `.jpeg`/`.jpg`/`.png` files.

2. **AnsweringAgent**: For each MCQ extracted, answer the question correctly and provide a brief explanation.

Format:
------------------------------------
Question: ...
Options: A..., B..., C..., D...
✅ Correct Answer: ...
Explanation: ...
------------------------------------
"""
)

User (to chat_manager):


Hi team. Here's the task:

1. **OCRAgent**: Run the function `extract_mcq_text` on the folder `/Users/naveenudayakumar/Documents/OpenAI/LLM_P3` and extract MCQ-style questions and options from all `.jpeg`/`.jpg`/`.png` files.

2. **AnsweringAgent**: For each MCQ extracted, answer the question correctly and provide a brief explanation.

Format:
------------------------------------
Question: ...
Options: A..., B..., C..., D...
✅ Correct Answer: ...
Explanation: ...
------------------------------------


--------------------------------------------------------------------------------

Next speaker: OCRAgent

OCRAgent (to chat_manager):

To accomplish this task, we can follow these steps:
1. Use Optical Character Recognition (OCR) to extract text from the images in the specified folder.
2. Identify and extract MCQ-style questions and answer options from the extracted text.
3. Answer the questions correctly and provide a brief explanation.

Let's start by creating 

ChatResult(chat_id=None, chat_history=[{'content': "\nHi team. Here's the task:\n\n1. **OCRAgent**: Run the function `extract_mcq_text` on the folder `/Users/naveenudayakumar/Documents/OpenAI/LLM_P3` and extract MCQ-style questions and options from all `.jpeg`/`.jpg`/`.png` files.\n\n2. **AnsweringAgent**: For each MCQ extracted, answer the question correctly and provide a brief explanation.\n\nFormat:\n------------------------------------\nQuestion: ...\nOptions: A..., B..., C..., D...\n✅ Correct Answer: ...\nExplanation: ...\n------------------------------------\n", 'role': 'assistant', 'name': 'User'}, {'content': "To accomplish this task, we can follow these steps:\n1. Use Optical Character Recognition (OCR) to extract text from the images in the specified folder.\n2. Identify and extract MCQ-style questions and answer options from the extracted text.\n3. Answer the questions correctly and provide a brief explanation.\n\nLet's start by creating a Python script to handle this task. 

In [27]:
import os
import pytesseract
from PIL import Image

# Function to extract MCQ-style text from an image
def extract_mcq_text(image_path):
    text = pytesseract.image_to_string(Image.open(image_path))
    # Implement logic to extract MCQ-style questions and options
    # For simplicity, let's assume that the questions have a '?' at the end and options are listed with A, B, C, D
    
    mcq_list = []
    current_mcq = {"question": "", "options": []}
    
    lines = [line.strip() for line in text.split("\n") if line.strip()]
    
    for line in lines:
        if line.endswith("?"):
            if current_mcq["question"]:
                mcq_list.append(current_mcq)
            current_mcq = {"question": line, "options": []}
        elif line.lower().startswith("a) ") or line.lower().startswith("a."):
            current_mcq["options"].append(line)
    
    mcq_list.append(current_mcq)  # Add the last MCQ
    
    return mcq_list

# Define the folder path
folder_path = "/Users/naveenudayakumar/Documents/OpenAI/LLM_P3"

# Process all image files in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".jpeg", ".jpg", ".png")):
        image_path = os.path.join(folder_path, filename)
        mcqs = extract_mcq_text(image_path)
        
        for mcq in mcqs:
            # Answer the question and provide an explanation
            # You can use additional logic here to answer the questions
            correct_answer = "B"  # Assuming the correct answer is always B for demonstration
            explanation = "Just an example explanation."
            
            print("------------------------------------")
            print("Question:", mcq["question"])
            print("Options:", ", ".join(mcq["options"]))
            print("✅ Correct Answer:", correct_answer)
            print("Explanation:", explanation)
            print("------------------------------------")

In [28]:
extract_mcq_text()

TypeError: extract_mcq_text() missing 1 required positional argument: 'image_path'